In [5]:
# --- 最终步骤：修正并完成情感数据合并 ---

import pandas as pd
import numpy as np

# 1. 加载已清理好的主数据集
# 这个文件是上一步运行后生成的，已经没有_x, _y后缀了
master_df = pd.read_csv('data/finalized/final_master_dataset.csv', parse_dates=['Date'])


# 2. 加载情感分析数据 (使用正确的列名)
try:
    # 使用正确的小写 'date' 来解析日期
    sentiment_df = pd.read_csv('data/processed/sentiment_scores.csv', parse_dates=['date'])

    # 3. 重命名列以匹配主DataFrame，为合并做准备
    sentiment_df.rename(columns={'date': 'Date', 'ticker': 'Ticker'}, inplace=True)
    
    print("成功加载并重命名情感数据列名。")
    print("'sentiment_scores.csv' 处理后的前5行:")
    print(sentiment_df.head())

except Exception as e:
    print(f"\n读取或处理情感分数文件时出错: {e}")
    sentiment_df = pd.DataFrame() # 创建空DataFrame以防万一


# 4. 将情感分数合并到主数据中 (如果文件成功加载)
if not sentiment_df.empty:
    # 合并键现在是 ['Date', 'Ticker']，两边都匹配了
    master_df = pd.merge(master_df, sentiment_df, on=['Date', 'Ticker'], how='left')

    # 5. 处理合并后可能产生的NaN值
    # 对于情感分数和文章计数，如果在某一天没有新闻，它的值会是NaN。
    # 我们用0来填充，代表中性情绪或0篇文章。
    # 使用文件中实际的列名
    sentiment_cols_to_fill = ['vader_sentiment_mean', 'finbert_sentiment_mean', 'article_count']
    
    for col in sentiment_cols_to_fill:
        if col in master_df.columns:
            master_df[col].fillna(0, inplace=True)

    print("\n已成功合并情感数据！")


# 6. 再次检查最终数据集的头部和列名
print("\n最终合并后的数据前5行:")
print(master_df.head())

print("\n最终数据集的列名:")
print(master_df.columns.tolist()) # 使用tolist()让输出更清晰

# 7. 保存最终的、完整的、包含所有特征的主数据集
master_df.to_csv('data/finalized/final_master_dataset.csv', index=False)

print("\n🎉🎉🎉 最终决战胜利！包含所有特征的完整主数据集 final_master_dataset.csv 已成功生成！")

C:\Users\ROG\AppData\Local\Temp\ipykernel_63624\1222971884.py:8: DtypeWarning: Columns (56,57,60,78,140,231,234,238,239,240) have mixed types. Specify dtype option on import or set low_memory=False.
  master_df = pd.read_csv('data/finalized/final_master_dataset.csv', parse_dates=['Date'])


成功加载并重命名情感数据列名。
'sentiment_scores.csv' 处理后的前5行:
  Ticker       Date  vader_sentiment_mean  finbert_sentiment_mean  \
0   AAPL 2025-10-31              0.186738                0.191549   
1   AAPL 2025-11-02              0.161408               -0.163420   
2   AAPL 2025-11-03              0.146783               -0.027235   
3   AAPL 2025-11-04             -0.039878               -0.211465   
4   AAPL 2025-11-05              0.013078               -0.293637   

   article_count  
0             26  
1             12  
2             18  
3              9  
4              9  

已成功合并情感数据！

最终合并后的数据前5行:
        Date Ticker    Open_x    High_x     Low_x   Close_x   Volume_x  \
0 2000-03-14   AAPL  0.909750  0.932500  0.855573  0.857449  428579200   
1 2000-03-15   AAPL  0.867769  0.902480  0.856511  0.872459  443609600   
2 2000-03-16   AAPL  0.880434  0.915614  0.859325  0.912330  378100800   
3 2000-03-17   AAPL  0.901542  0.938128  0.897789  0.938128  305043200   
4 2000-03-20   AAPL  0.9268

In [6]:
import pandas as pd
import numpy as np
import os

# --- 准备工作：加载合并后的主数据集 ---

# 【已修正】将文件路径从 'data/final/' 修改为 'data/finalized/' 以匹配您的文件结构
file_path = 'data/finalized/final_master_dataset.csv'

# 检查文件是否存在
if not os.path.exists(file_path):
    print(f"错误：找不到文件 '{file_path}'。请再次检查您的文件路径是否正确。")
else:
    # 加载数据集，并将'Date'列解析为日期时间对象
    master_df = pd.read_csv(file_path, parse_dates=['Date'])
    
    # 为保证操作正确，最好按股票代码和日期排序
    master_df.sort_values(by=['Ticker', 'Date'], inplace=True)
    master_df.reset_index(drop=True, inplace=True)
    
    print("主数据集加载成功，数据预览：")
    print(master_df.head())
    print("\n" + "="*50 + "\n")


    # --- 任务 1: 创建滞后特征 (Create Lagged Features) ---
    
    print("步骤 1: 开始创建滞后特征...")
    
    # 识别特征列和目标列
    # 目标列通常是未来的收益率
    target_col = 'target_1d_return' 
    
    # 标识符列，不应作为特征
    id_cols = ['Date', 'Ticker']
    
    # 特征列是除了目标列和标识符列之外的所有列
    # 注意：如果您的数据中有 'Unnamed: 0' 这样的列，也需要在这里排除
    feature_cols = [col for col in master_df.columns if col not in [target_col] + id_cols and 'Unnamed' not in col]
    
    print(f"找到 {len(feature_cols)} 个原始特征。")
    
    # 核心操作：对每个股票分组，然后对特征列进行 shift(1) 操作
    # 这确保了我们用 T-1 天的特征来匹配 T 天的目标，并且不会将一只股票的数据泄露给另一只
    lagged_features = master_df.groupby('Ticker')[feature_cols].shift(1)
    
    # 为了清晰起见，给滞后的特征列名添加后缀 '_lag1'
    lagged_features.columns = [f'{col}_lag1' for col in feature_cols]
    
    # 将原始的标识符、目标列与新生成的滞后特征合并
    processed_df = pd.concat([master_df[id_cols + [target_col]], lagged_features], axis=1)
    
    # 清理数据：shift 操作会在每个股票的开头产生一行 NaN，这些行没有历史数据，必须删除
    initial_rows = len(processed_df)
    processed_df.dropna(inplace=True)
    final_rows = len(processed_df)
    
    print(f"创建滞后特征完成。因 shift 操作删除了 {initial_rows - final_rows} 行数据。")
    print("处理后的数据预览（特征均已滞后）：")
    print(processed_df.head())
    print("\n" + "="*50 + "\n")

    
    # --- 任务 2: 定义特征 (X) 和目标 (y) ---
    
    print("步骤 2: 定义特征矩阵 X 和目标向量 y...")
    
    # 特征 X 是所有以 '_lag1' 结尾的列
    feature_cols_lagged = [col for col in processed_df.columns if col.endswith('_lag1')]
    X = processed_df[feature_cols_lagged]
    
    # 目标 y 是我们的预测目标列
    y = processed_df[target_col]
    
    print(f"特征矩阵 X 的维度: {X.shape}")
    print(f"目标向量 y 的维度: {y.shape}")
    print("\n" + "="*50 + "\n")

    
    # --- 任务 3: 时序感知的训练/验证/测试集划分 ---
    
    print("步骤 3: 开始进行时序感知的训练/验证/测试集划分...")
    
    # 我们将严格按照时间顺序划分数据集，以模拟真实预测流程
    # 训练集: 截止到 2022 年底的数据
    # 验证集: 2023 年全年的数据
    # 测试集: 2024 年及之后的数据
    # 您可以根据数据的实际时间范围调整这些日期
    train_end_date = '2022-12-31'
    validation_end_date = '2023-12-31'
    
    # 获取与 X 和 y 对应的日期信息用于过滤
    dates = processed_df['Date']
    
    # 创建布尔掩码 (boolean masks) 来选择数据
    train_mask = (dates <= train_end_date)
    validation_mask = (dates > train_end_date) & (dates <= validation_end_date)
    test_mask = (dates > validation_end_date)
    
    # 应用掩码进行分割
    X_train, y_train = X[train_mask], y[train_mask]
    X_val, y_val = X[validation_mask], y[validation_mask]
    X_test, y_test = X[test_mask], y[test_mask]
    
    # 打印划分结果以供验证
    print("数据集划分结果:")
    print("-" * 30)
    print(f"训练集 (Train) 样本数: {len(X_train)}")
    if not X_train.empty:
        print(f"训练集时间范围: {processed_df.loc[train_mask, 'Date'].min().date()} to {processed_df.loc[train_mask, 'Date'].max().date()}")
    
    print("-" * 30)
    print(f"验证集 (Validation) 样本数: {len(X_val)}")
    if not X_val.empty:
        print(f"验证集时间范围: {processed_df.loc[validation_mask, 'Date'].min().date()} to {processed_df.loc[validation_mask, 'Date'].max().date()}")

    print("-" * 30)
    print(f"测试集 (Test) 样本数: {len(X_test)}")
    if not X_test.empty:
        print(f"测试集时间范围: {processed_df.loc[test_mask, 'Date'].min().date()} to {processed_df.loc[test_mask, 'Date'].max().date()}")
    print("-" * 30)

    # 检查分割是否正确（无重叠且按时间排序）
    if not X_train.empty and not X_val.empty:
        # 仅在训练集和验证集非空时检查
        if X_val.index.min() > X_train.index.max():
             print("\n数据分割检查通过：训练集和验证集按时间顺序排列，无重叠。")
        if not X_test.empty:
            # 仅在测试集也非空时检查
            if X_test.index.min() > X_val.index.max():
                print("数据分割检查通过：验证集和测试集按时间顺序排列，无重叠。")
            else:
                print("\n警告：验证集和测试集之间存在索引重叠或顺序错误。")
        else:
            print("\n提示：测试集为空，这可能是因为您的数据时间范围未达到 2024 年。")
    else:
        print("\n警告：一个或多个数据集为空，请检查您的日期范围和数据。")
        
    print("\n" + "="*50 + "\n")
    
    # --- 里程碑: 保存处理好的数据集 ---
    
    print("里程碑任务: 保存 X 和 y 的各个部分到 CSV 文件...")
    
    # 创建目录（如果尚不存在）
    # 这个目录 'data/final/model_input' 将被自动创建
    output_dir = 'data/final/model_input'
    os.makedirs(output_dir, exist_ok=True)
    
    # 保存文件
    X_train.to_csv(os.path.join(output_dir, 'X_train.csv'), index=False)
    y_train.to_csv(os.path.join(output_dir, 'y_train.csv'), index=False, header=True)
    
    X_val.to_csv(os.path.join(output_dir, 'X_val.csv'), index=False)
    y_val.to_csv(os.path.join(output_dir, 'y_val.csv'), index=False, header=True)
    
    X_test.to_csv(os.path.join(output_dir, 'X_test.csv'), index=False)
    y_test.to_csv(os.path.join(output_dir, 'y_test.csv'), index=False, header=True)
    
    print(f"所有模型输入文件已成功保存到 '{output_dir}' 目录下。")

C:\Users\ROG\AppData\Local\Temp\ipykernel_63624\1728895391.py:15: DtypeWarning: Columns (56,57,60,78,140,231,234,238,239,240) have mixed types. Specify dtype option on import or set low_memory=False.
  master_df = pd.read_csv(file_path, parse_dates=['Date'])


主数据集加载成功，数据预览：
        Date Ticker    Open_x    High_x     Low_x   Close_x   Volume_x  \
0 2000-03-14   AAPL  0.909750  0.932500  0.855573  0.857449  428579200   
1 2000-03-15   AAPL  0.867769  0.902480  0.856511  0.872459  443609600   
2 2000-03-16   AAPL  0.880434  0.915614  0.859325  0.912330  378100800   
3 2000-03-17   AAPL  0.901542  0.938128  0.897789  0.938128  305043200   
4 2000-03-20   AAPL  0.926872  0.947510  0.918428  0.923118  204489600   

   SMA_20_x    SMA_50    EMA_20  ...  ipoExpectedDate  dividendRate  \
0  0.892049  0.830715  0.889972  ...              NaN          1.04   
1  0.891017  0.831363  0.888304  ...              NaN          1.04   
2  0.893808  0.834224  0.890592  ...              NaN          1.04   
3  0.897607  0.837376  0.895119  ...              NaN          1.04   
4  0.902016  0.841579  0.897786  ...              NaN          1.04   

   dividendYield  dividendDate  address2  prevName  nameChangeDate  \
0           0.38  1.762992e+09       NaN   